In [1]:
import torch
import numpy as np
from torch_geometric.data import Data
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.graphs import construct_graph
from graphein.protein.edges.distance import add_distance_threshold
from functools import partial
import esm
import networkx as nx

# Load ESM-2 model
protein_model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
protein_model.eval()

# Configure Graphein
config = ProteinGraphConfig(
    edge_construction_functions=[
        partial(add_distance_threshold, long_interaction_threshold=0, threshold=8)
    ]
)

def get_esm2_embedding(seq, pdb_id):
    # Handle sequences longer than 1022 residues
    seq_feat = []
    for i in range(0, len(seq), 1022):
        data = [(pdb_id, seq[i:i+1022])]
        batch_labels, batch_strs, batch_tokens = batch_converter(data)
        with torch.no_grad():
            results = protein_model(batch_tokens, repr_layers=[33], return_contacts=True)
        token_representations = results["representations"][33]
        seq_feat.append(token_representations.squeeze(0)[1:len(data[0][1])+1])
    seq_feat = torch.cat(seq_feat, dim=0)
    
    return seq_feat

def adj2edge_index(adj):
    # Convert adjacency matrix to edge index format
    return adj.nonzero().t().contiguous()

def protein_graph(pdb_path, pdb_id):
    # Construct graph using Graphein
    try:
        g = construct_graph(config=config, path=pdb_path)
    except Exception as e:
        print(f"Error constructing graph for {pdb_id}: {e}")
        return None
    
    # Extract sequence
    seq = ''.join(g.graph[key] for key in g.graph.keys() if key.startswith("sequence_"))
    
    # Get node features using ESM-2
    node_feat = get_esm2_embedding(seq, pdb_id)
    
    # Build edges
    A = nx.to_numpy_array(g, nonedge=0, weight='distance')
    edge_index = adj2edge_index(torch.tensor(A))
    
    # Add extra structural information
    node_ids = list(g.nodes())
    extra_features = torch.tensor([
        [g.nodes[node_id].get('is_helix', 0),
         g.nodes[node_id].get('is_sheet', 0),
         g.nodes[node_id].get('is_turn', 0),
         g.nodes[node_id].get('solvent_accessibility', 0)]
        for node_id in node_ids
    ])
    
    node_feat = torch.cat([node_feat, extra_features], dim=1)
    
    # Create Data object
    pos = torch.tensor([g.nodes[node_id]['coords'] for node_id in node_ids])
    data = Data(x=node_feat, edge_index=edge_index, pos=pos)
    
    return data

# Usage example
if __name__ == "__main__":
    pdb_path = "./../Testset/4ytc/4ytc_protein.pdb"
    pdb_id = "4ytc"  # Example PDB ID
    
    protein_graph_data = protein_graph(pdb_path, pdb_id)
    
    if protein_graph_data is not None:
        print(f"Protein {pdb_id}:")
        print(f"Node features: {protein_graph_data.x.shape}")
        print(f"Edge index: {protein_graph_data.edge_index.shape}")
        print(f"3D coordinates: {protein_graph_data.pos.shape}")
    else:
        print(f"Failed to create graph for {pdb_id}")

[09/09/24 18:58:39] WARNING  To use the Graphein submodule                                         ]8;id=461368;file:///mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphein/protein/features/sequence/embeddings.py\embeddings.py]8;;\:]8;id=350981;file:///mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphein/protein/features/sequence/embeddings.py#45\45]8;;\
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             biovec                                                                                
                             biovec cannot be installed via conda                                                  
                             Alternatively, you can install graphein with the extras:                              
                                                                                                                   
                             pip install graphein[extras]                                                          

Output()

[09/09/24 18:58:42] DEBUG    Deprotonating protein. This removes H atoms from the pdb_df dataframe    ]8;id=16976;file:///mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=660322;file:///mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphein/protein/graphs.py#188\188]8;;\

                    DEBUG    Detected 284 total nodes                                                 ]8;id=871979;file:///mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=267225;file:///mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphein/protein/graphs.py#435\435]8;;\

                    DEBUG    Reading meiler embeddings from:                                       ]8;id=786987;file:///mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphein/protein/features/nodes/amino_acid.py\amino_acid.py]8;;\:]8;id=79768;file:///mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphein/protein/features/nodes/amino_acid.py#58\58]8;;\
                             /mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphei                 
                             n/protein/features/nodes/meiler_embeddings.csv                                        

                    INFO     Found: 2912 distance edges                                             ]8;id=274335;file:///mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphein/protein/edges/distance.py\distance.py]8;;\:]8;id=700285;file:///mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphein/protein/edges/distance.py#964\964]8;;\

                    INFO     Added 2912 distance edges. (0            removed by LIN)               ]8;id=83582;file:///mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphein/protein/edges/distance.py\distance.py]8;;\:]8;id=925850;file:///mnt/data/miniforge3/envs/onebind/lib/python3.9/site-packages/graphein/protein/edges/distance.py#983\983]8;;\

Protein 4ytc:
Node features: torch.Size([284, 1284])
Edge index: torch.Size([2, 2628])
3D coordinates: torch.Size([284, 3])


/tmp/ipykernel_3446583/2756297163.py:72: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1678402374358/work/torch/csrc/utils/tensor_new.cpp:245.)
  pos = torch.tensor([g.nodes[node_id]['coords'] for node_id in node_ids])
